In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Column names for question items
ori_responses = ["SAT_1", "SAT_2", "SAT_3", "LOY_1", "LOY_2", "LOY_3", "TRUST_1", "TRUST_2", "TRUST_3", "TRUST_4"]
target_df = pd.read_excel('DataInBrief_Bankdata.xlsx')
selected_columns_df = target_df[ori_responses]
base_path = './Data_Case2' ## example path for given result data we provided


In [3]:
distance_dict = {
    1: -1, 
    2: -1, 
    3: -1, 
    4: 0, 
    5: 1, 
    6: 1, 
    7: 1
}

In [4]:
results = []

for folder in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder)
    if os.path.isdir(folder_path):
        llm_file = os.path.join(folder_path, 'llm_responses.csv')
        llm_df = pd.read_csv(llm_file, index_col=0)
        print(f"==========Results of {folder}===========\n")

        original_df_mapped = selected_columns_df.applymap(lambda x: distance_dict.get(x, None))
        llm_df_mapped = llm_df.applymap(lambda x: distance_dict.get(x, None))
        if list(original_df_mapped.index) != list(llm_df_mapped.index):
            print("Error!")
        accuracies = []
        for i in range(1, 11):
            llm_df_mapped[f'Q{i}_original'] = original_df_mapped.iloc[:, i - 1]
            llm_df_mapped[f'Q{i}_check'] = np.where(llm_df_mapped[f'Q{i}_original'] == llm_df_mapped[f'Q{i}'], 1, 0)
            accuracy = llm_df_mapped[f'Q{i}_check'].mean() * 100
            accuracies.append(accuracy)
        mean_accuracy = sum(accuracies) / len(accuracies)
        
        print(f"Q1 Accuracy: {accuracies[0]:.2f}%")
        print(f"Q2 Accuracy: {accuracies[1]:.2f}%")
        print(f"Q3 Accuracy: {accuracies[2]:.2f}%")
        print(f"Q4 Accuracy: {accuracies[3]:.2f}%")
        print(f"Q5 Accuracy: {accuracies[4]:.2f}%")
        print(f"Q6 Accuracy: {accuracies[5]:.2f}%")
        print(f"Q7 Accuracy: {accuracies[6]:.2f}%")
        print(f"Q8 Accuracy: {accuracies[7]:.2f}%")
        print(f"Q9 Accuracy: {accuracies[8]:.2f}%")
        print(f"Q10 Accuracy: {accuracies[9]:.2f}%")
        print(f"Mean Accuracy: {mean_accuracy:.2f}%\n")

        results.append({
            'Folder': folder,
            'Q1_Accuracy': accuracies[0],
            'Q2_Accuracy': accuracies[1],
            'Q3_Accuracy': accuracies[2],
            'Q4_Accuracy': accuracies[3],
            'Q5_Accuracy': accuracies[4],
            'Q6_Accuracy': accuracies[5],
            'Q7_Accuracy': accuracies[6],
            'Q8_Accuracy': accuracies[7],
            'Q9_Accuracy': accuracies[8],
            'Q10_Accuracy': accuracies[9],
            'Mean_Accuracy': mean_accuracy
        })

==========Results of Baseline_prompting===========

Q1 Accuracy: 82.52%
Q2 Accuracy: 82.07%
Q3 Accuracy: 75.26%
Q4 Accuracy: 84.89%
Q5 Accuracy: 44.15%
Q6 Accuracy: 37.48%
Q7 Accuracy: 47.41%
Q8 Accuracy: 42.07%
Q9 Accuracy: 50.52%
Q10 Accuracy: 49.48%
Mean Accuracy: 59.59%

==========Results of Demo_prompting===========

Q1 Accuracy: 80.00%
Q2 Accuracy: 78.96%
Q3 Accuracy: 55.11%
Q4 Accuracy: 84.44%
Q5 Accuracy: 33.04%
Q6 Accuracy: 38.22%
Q7 Accuracy: 41.48%
Q8 Accuracy: 38.37%
Q9 Accuracy: 37.04%
Q10 Accuracy: 38.96%
Mean Accuracy: 52.56%

==========Results of Omni_prompting===========

Q1 Accuracy: 86.37%
Q2 Accuracy: 85.04%
Q3 Accuracy: 85.04%
Q4 Accuracy: 83.85%
Q5 Accuracy: 62.22%
Q6 Accuracy: 45.04%
Q7 Accuracy: 70.67%
Q8 Accuracy: 71.56%
Q9 Accuracy: 66.81%
Q10 Accuracy: 73.48%
Mean Accuracy: 73.01%

==========Results of Persona_Driven_prompting===========

Q1 Accuracy: 84.74%
Q2 Accuracy: 84.74%
Q3 Accuracy: 79.26%
Q4 Accuracy: 83.70%
Q5 Accuracy: 58.81%
Q6 Accuracy: 43.70%
Q7

In [5]:
save_path = './analysis_result/appendix/consistency_analysis_case2/'
if not os.path.exists(save_path):
    os.makedirs(save_path)

results_df = pd.DataFrame(results)
results_df.to_csv('./analysis_result/appendix/consistency_analysis_case2/result.csv', index=False)